# Mass spectrometry data

The objective of this exercise is to read in raw peptide MSMS spectrum information and output a dataframe.
The .msp file can be downloaded [here](https://chemdata.nist.gov/download/peptide_library/libraries/cptaclib/2015/cptac2_mouse_hcd_selected.msp.tar.gz).

The information in this ASCII based text file is organized spectrum by spectrum.
The first line per spectrum provides formatted like this:

&emsp;<code>Name: sequence/charge_nmods_collisionenergy</code>

followed by a comment section which can be disregarded and the actual spectrum data which is tab-separated:

&emsp;<code>m/z&emsp;intensity&emsp;additional_info</code>

Spectra are separated by an empty line.

Code a function that returns two DataFrames or arrays containing the processed and filtered data. The first one should contain the spectrum information (n_spectra, n_m/z_features) and the second one the sequences per row (n_spectra).

Here are some general guidelines:

* The m/z values need to be binned to integer values (mathematically rounded), otherwise the dataframe size would get out of hand. This will allow for multiple values mapped to a single bin (e.g. if there are peaks at 145.1 and 145.2). Here, only the maximum of those peaks should be kept in the final dataframe.

* Rows that are all-zero should be dropped.

Your function should allow for selecting a range on the x-axis (m/z-range). All peaks outside this range can be disregarded. Furthermore, only spectra within a set collision energy range and a maximum sequence length should be contained in the output dataframe.

The faster your function runs, the better. I will time them all in the end.

In [1]:
import numpy as np
import pandas as pd
import timeit

In [2]:
def parse_name_line(file_line):
    parsed_name = file_line[6:]
    parsed_seq = file_line[6:].split('/')[0]
    parsed_ce = float(file_line[6:].split('/')[1].split('_')[-1][:-2])
    return parsed_name, parsed_seq, parsed_ce
    

In [3]:
def consider_conditions(seq, ce, max_seq_len, max_ce, min_ce):
    condition = True
    if len(seq) > max_seq_len:
        condition = False
    if max_ce <= ce or ce <= min_ce:
        condition = False
    return condition

In [4]:
def parse_peak_data(line):
    parsed_line = line.split('\t')
    peak_pos = round(float(parsed_line[0]))
    peak_intensity = float(parsed_line[1])
    return peak_pos, peak_intensity

In [7]:
def msp_to_df(
    input_file,
    max_seq_len=30,
    min_ce=36,
    max_ce=40,
    mz_min=135,
    mz_max=1400,
):
    """
    Function to read spectrum data from .msp file and convert to dataframe.
    Args:
        input_file (str): path to .msp file
        max_seq_len (int): maximum acceptable sequence length
        min_ce (int): minimum collision energy of spectra to be included in df
        max_ce (int): maximum collision energy of spectra to be included in df
        mz_min (int): lower boundary for m/z to be included in df
        mz_max (int): lower boundary for m/z to be included in df

    Returns:
        df (pd.DataFrame or np.array): spectrum information within defined parameters [n_spectra, n_features]
        seqs (pd.DataFrame or np.array): sequences
    """
    col_names = ["Name", "Sequence"] + list(np.arange(mz_min,mz_max+1,1))
    ms_df = pd.DataFrame(columns = col_names)
    
    counter = 0
    conditions = False
    with open(input_file, 'r') as file:

        for line in file:
            
            line = line.strip()

            if line[:5] == 'Name:':
                conditions = False
                name, seq, ce = parse_name_line(line)
                conditions = consider_conditions(seq, ce, max_seq_len, max_ce, min_ce)

                if conditions == True:
                    new_df_line = pd.Series(0, index=ms_df.columns)
                    ms_df = ms_df.append(new_df_line, ignore_index=True)
                    ms_df.at[counter, "Name"] = name
                    ms_df.at[counter, "Sequence"] = seq
                    counter += 1

            if conditions == True:
                if line[0:1] in np.arange(10).astype('str'): # line with peaks begins with a number
                    position, intensity = parse_peak_data(line)

                    if mz_min < position < mz_max:
                        
                        if ms_df.at[counter-1, position] < intensity:
                            ms_df.at[counter-1, position] = intensity
                            
    ms_df = ms_df.loc[(ms_df.iloc[:, 2:] != 0).any(axis=1)]
    
    return ms_df

In [9]:
dataframe = msp_to_df('..\..\data\cptac2_mouse_hcd_selected.msp')
dataframe.head()

,Name,Sequence,135,136,137,138,139,140,141,142,...,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400
0,AAAFEDQENETVVVK/2_0_38.7eV,AAAFEDQENETVVVK,0,2851.7,0,0,0,0,1404.0,0,...,0,0,0,0,0,0,0,0,2797.6,0
1,AAGTIQTSVQEVNSK/2_0_37.3eV,AAGTIQTSVQEVNSK,0,2626.0,0,0,1553.4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AAGVSVEPFWPGLFAK/2_0_39.3eV,AAGVSVEPFWPGLFAK,0,59857.3,8295.8,0,3220.5,0,24351.6,2054.0,...,0,0,0,0,0,0,0,0,0,0
3,"AAHSQCAYSNPEGTVLLACEESR/3_2(5,C,CAM)(18,C,CAM)...",AAHSQCAYSNPEGTVLLACEESR,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"AAHSQCAYSNPEGTVLLACEESR/3_2(5,C,CAM)(18,C,CAM)...",AAHSQCAYSNPEGTVLLACEESR,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1946.0,0,0,0
